In [1]:
import torch
import torch.nn as nn
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

# 1x32x32 input --> (5x5) kernel with stride (1,1)
# 6x28x28 input --> avg poll with stride(2,2)
# 6x14x14 input --> (5x5) kernel with stride(1,1)
# 16x1010 input --> avg pool with stride(2,2)

#Conv 5x5 to 120 channels x Linear 120 -> 84x Liner 10




In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.relu = nn.ReLU()
        
        self.pool = nn.AvgPool2d(kernel_size=(2,2),stride=(2,2))
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6,kernel_size=(5,5), stride=(1, 1), padding=(2,2))
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16,kernel_size=(5,5), stride=(1, 1), padding=(0,0))
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120,kernel_size=(5,5), stride=(1, 1), padding=(0,0))
        self.linear1 = nn.Linear(120,84)
        self.linear2 = nn.Linear(84,10)
        
    def forward(self,x):

        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = x.reshape(x.shape[0],-1)
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x
    
#x = torch.randn(64,1,32,32)
#model = LeNet()
#print(model(x).shape)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size =256
in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 3

In [4]:
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize network
model = LeNet().to(device)

C:\Users\emrea\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        #data = data.reshape(data.shape[0],-1)
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)


    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")
torch.save(model.state_dict(), "model.pth")

100%|██████████| 938/938 [00:13<00:00, 68.47it/s]


Accuracy on training set: 97.94
Accuracy on test set: 97.87
